# Querying postgreSQL in Jupyter notebook for teachback

Notebook for 4/9/20 teachback session by Ben. The notebook format is useful for writing notes and iterating over SQL queries. You can look at the "hard" examples down below to show how queries can be broken down into smaller parts and then combined into a more complicated query.

In [2]:
import pandas as pd
import sqlalchemy
import sqlalchemy_utils
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [4]:
# Alise's database - ecommerce
# Working with PostgreSQL in Python
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(host = 'ec2-54-245-31-214.us-west-2.compute.amazonaws.com',
                       port = '5291',
                       database = 'ecommerce',
                       user = 'sqlpractice',
                       password = 'iloveSQL!')

# Notes

## Order of execution

1. FROM, including JOINs
2. WHERE
3. GROUP BY
4. HAVING
5. WINDOW functions
6. SELECT
7. DISTINCT
8. UNION
9. ORDER BY
10. LIMIT and OFFSET

It can be summarized as:
**calling -> aggregating -> displaying -> filter**

- First creates a working dataset, then filters with rows added by conditions
- Column name aliases are not accessed for all commands except after SELECT (aliases for table names are okay)

## Window functions

- Window functions (always in SELECT statement; it can also be in ORDER BY)
    - Window functions can contain multiple types of functions, including aggregate, rank 
    
- Window functions
    - RANK() 
    - DENSE_RANK() - no gaps in rank values
    - ROW_NUMBER() - assign a unique sequential interger to rows within a partition of a result set, the first row starts with 1
    - NTILE() - to identify percentile or quartile
    - LAG() - pulls from previous row
    - LEAD() - pulls from following row
    

## Other topics to study

- Self joins
- Cross joins
- Data types
- DATES/DATETIME
- Built-in SQL functions
    - ROUND()
    - CAST() - moves something into a float to allow division - see above
- Creating and updating tables
- Pivoting tables with CASE statements

# Queries

## Overview of tables

Set to ecommerce database

In [4]:
## Overview of tables - 
sql_query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public'
AND table_type='BASE TABLE';
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

table_name
0               categories
1  customergroupthresholds
2                customers
3                 products
4                suppliers
5                employees
6             orderdetails
7                   orders
8                 shippers

In [5]:
## Overview of table
sql_query = """
SELECT *
FROM categories
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

categoryid categoryname                                        description
0           1    Beverages        Soft drinks, coffees, teas, beers, and ales
1           2   Condiments  Sweet and savory sauces, relishes, spreads, an...
2           3  Confections                Desserts, candies, and sweet breads

In [6]:
## Overview of table
sql_query = """
SELECT *
FROM customergroupthresholds
LIMIT 5;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customergroupname rangebottom                 rangetop
0               Low       $0.00                $1,000.00
1            Medium   $1,000.00                $5,000.00
2              High   $5,000.00               $10,000.00
3         Very High  $10,000.00  $922,337,203,685,477.58

In [7]:
## Overview of table
sql_query = """
SELECT *
FROM customers
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customerid                         companyname     contactname  \
0      ALFKI                 Alfreds Futterkiste    Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados    Ana Trujillo   
2      ANTON              Antonio Moreno Taquera  Antonio Moreno   

           contacttitle                       address        city region  \
0  Sales Representative                 Obere Str. 57      Berlin   None   
1                 Owner  Avda. de la Constitucin 2222  Mxico D.F.   None   
2                 Owner               Mataderos  2312  Mxico D.F.   None   

  postalcode  country         phone           fax  
0      12209  Germany   030-0074321   030-0076545  
1      05021   Mexico  (5) 555-4729  (5) 555-3745  
2      05023   Mexico  (5) 555-3932          None

In [8]:
## Overview of table
sql_query = """
SELECT *
FROM products
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

productid    productname  supplierid  categoryid      quantityperunit  \
0          1           Chai           1           1   10 boxes x 20 bags   
1          2          Chang           1           1   24 - 12 oz bottles   
2          3  Aniseed Syrup           1           2  12 - 550 ml bottles   

  unitprice  unitsinstock  unitsonorder  reorderlevel  discontinued  
0    $18.00            39             0            10         False  
1    $19.00            17            40            25         False  
2    $10.00            13            70            25         False

In [9]:
## Overview of table
sql_query = """
SELECT *
FROM suppliers
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

supplierid                 companyname       contactname  \
0      1                  Exotic Liquids  Charlotte Cooper   
1      2      New Orleans Cajun Delights     Shelley Burke   
2      3       Grandma Kelly's Homestead     Regina Murphy   

           contacttitle         address         city region postalcode  \
0    Purchasing Manager  49 Gilbert St.       London   None    EC1 4SD   
1  orders Administrator  P.O. Box 78934  New Orleans     LA      70117   
2  Sales Representative  707 Oxford Rd.    Ann Arbor     MI      48104   

  country           phone             fax     homepage  
0      UK  (171) 555-2222            None         None  
1     USA  (100) 555-4822            None  #CAJUN.HTM#  
2     USA  (313) 555-5735  (313) 555-3349         None

In [10]:
## Overview of table
sql_query = """
SELECT *
FROM employees
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

employeeid   lastname firstname                  title titleofcourtesy  \
0           1    Davolio     Nancy   Sales Representative             Ms.   
1           2     Fuller    Andrew  Vice President, Sales             Dr.   
2           3  Leverling     Janet   Sales Representative             Ms.   

   birthdate   hiredate                    address      city region  \
0 1966-12-08 2010-05-01  507 - 20th Ave. E. Apt.57  New York     NY   
1 1970-02-19 2010-08-14         908 W. Capital Way    Tacoma     WA   
2 1981-08-30 2010-04-01         722 Moss Bay Blvd.  Kirkland     WA   

  postalcode country       homephone extension  \
0      10027     USA  (206) 555-9857      5467   
1      98401     USA  (206) 555-9482      3457   
2      98033     USA  (206) 555-3412      3355   

                                               notes  reportsto  \
0  Education includes a BA in psychology from Col...        2.0   
1  Andrew received his BTS commercial in 1974 and...        NaN   
2  Janet has a BS degree in chemistry from Boston...        2.0   

                                photopath  
0    http://accweb/emmployees/davolio.bmp  
1     http://accweb/emmployees/fuller.bmp  
2  http://accweb/emmployees/leverling.bmp

In [11]:
## Overview of table
sql_query = """
SELECT *
FROM orderdetails
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount
0    10248         11    $14.00        12       0.0
1    10248         42     $9.80        10       0.0
2    10248         72    $34.80         5       0.0

In [12]:
## Overview of table
sql_query = """
SELECT *
FROM orders
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid customerid  employeeid           orderdate requireddate  \
0    10248      VINET           5 2014-07-04 08:00:00   2014-08-01   
1    10249      TOMSP           6 2014-07-05 04:00:00   2014-08-16   
2    10250      HANAR           4 2014-07-08 15:00:00   2014-08-05   

  shippeddate  shipvia freight                   shipname         shipaddress  \
0  2014-07-16        3  $32.38  Vins et alcools Chevalier  59 rue de l'Abbaye   
1  2014-07-10        1  $11.61          Toms Spezialitten       Luisenstr. 48   
2  2014-07-12        2  $65.83              Hanari Carnes      Rua do Pao, 67   

         shipcity shipregion shippostalcode shipcountry  
0           Reims       None          51100      France  
1          Mnster       None          44087     Germany  
2  Rio de Janeiro         RJ      05454-876      Brazil

In [13]:
## Overview of table
sql_query = """
SELECT *
FROM shippers
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

shipperid       companyname           phone
0          1    Speedy Express  (503) 555-9831
1          2    United Package  (503) 555-3199
2          3  Federal Shipping  (503) 555-9931

## Window functions: LAG() and LEAD()

SQL concepts: LAG, LEAD, subquery, order of operations

What is the difference in quantity ordered of a product from a previous order?

In [14]:
sql_query = """
SELECT *
FROM orderdetails
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount
0    10248         11    $14.00        12       0.0
1    10248         42     $9.80        10       0.0
2    10248         72    $34.80         5       0.0

In [15]:
# Limit to one product
sql_query = """
SELECT *
FROM orderdetails
WHERE productid=11
LIMIT 10;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount
0    10248         11    $14.00        12       0.0
1    10296         11    $16.80        12       0.0
2    10327         11    $16.80        50       0.2
3    10353         11    $16.80        12       0.2
4    10365         11    $16.80        24       0.0
5    10407         11    $16.80        30       0.0
6    10434         11    $16.80         6       0.0
7    10442         11    $16.80        30       0.0
8    10443         11    $16.80         6       0.2
9    10466         11    $16.80        10       0.0

NOTE FOR THE NEXT CELL: In postgreSQL, you can sometimes leave OVER() as empty. It would work in the query below. But I tried something in Leetcode where I left it empty and it gave me an error. I think for other SQL variants, you would have to put in an ORDER BY so that's why I'm including it here. Since LAG() and LEAD() are dependent on the table order, I think it's good practice to put this in. Thank you Mike for mentioning this.

In [19]:
# Show each individually - have to confirm that the orderid is in order
sql_query = """
SELECT *,
       LAG(quantity, 1) OVER(ORDER BY orderid) AS quantity_prev,
       LEAD(quantity, 1) OVER(ORDER BY orderid) AS quantity_ahead
FROM orderdetails
WHERE productid=11
LIMIT 10;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount  quantity_prev  \
0    10248         11    $14.00        12       0.0            NaN   
1    10296         11    $16.80        12       0.0           12.0   
2    10327         11    $16.80        50       0.2           12.0   
3    10353         11    $16.80        12       0.2           50.0   
4    10365         11    $16.80        24       0.0           12.0   
5    10407         11    $16.80        30       0.0           24.0   
6    10434         11    $16.80         6       0.0           30.0   
7    10442         11    $16.80        30       0.0            6.0   
8    10443         11    $16.80         6       0.2           30.0   
9    10466         11    $16.80        10       0.0            6.0   

   quantity_ahead  
0              12  
1              50  
2              12  
3              24  
4              30  
5               6  
6              30  
7               6  
8              10  
9               5

In [20]:
# Calculate difference
sql_query = """
SELECT t.*,
       t.quantity-t.quantity_prev AS q_diff_fr_prev,
       t.quantity-t.quantity_ahead AS q_diff_fr_ahead
FROM
    (SELECT *,
           LAG(quantity, 1) OVER(ORDER BY orderid) AS quantity_prev,
           LEAD(quantity, 1) OVER(ORDER BY orderid) AS quantity_ahead
    FROM orderdetails
    WHERE productid=11) AS t;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount  quantity_prev  \
0     10248         11    $14.00        12      0.00            NaN   
1     10296         11    $16.80        12      0.00           12.0   
2     10327         11    $16.80        50      0.20           12.0   
3     10353         11    $16.80        12      0.20           50.0   
4     10365         11    $16.80        24      0.00           12.0   
5     10407         11    $16.80        30      0.00           24.0   
6     10434         11    $16.80         6      0.00           30.0   
7     10442         11    $16.80        30      0.00            6.0   
8     10443         11    $16.80         6      0.20           30.0   
9     10466         11    $16.80        10      0.00            6.0   
10    10486         11    $16.80         5      0.00           10.0   
11    10489         11    $16.80        15      0.25            5.0   
12    10528         11    $21.00         3      0.00           15.0   
13    10535         11    $21.00        50      0.10            3.0   
14    10542         11    $21.00        15      0.05           50.0   
15    10545         11    $21.00        10      0.00           15.0   
16    10553         11    $21.00        15      0.00           10.0   
17    10566         11    $21.00        35      0.15           15.0   
18    10570         11    $21.00        15      0.05           35.0   
19    10614         11    $21.00        14      0.00           15.0   
20    10637         11    $21.00        10      0.00           14.0   
21    10698         11    $21.00        15      0.00           10.0   
22    10726         11    $21.00         5      0.00           15.0   
23    10770         11    $21.00        15      0.25            5.0   
24    10797         11    $21.00        20      0.00           15.0   
25    10800         11    $21.00        50      0.10           20.0   
26    10823         11    $21.00        20      0.10           50.0   
27    10842         11    $21.00        15      0.00           20.0   
28    10862         11    $21.00        25      0.00           15.0   
29    10869         11    $21.00        10      0.00           25.0   
30    10889         11    $21.00        40      0.00           10.0   
31    10912         11    $21.00        40      0.25           40.0   
32    10926         11    $21.00         2      0.00           40.0   
33    10944         11    $21.00         5      0.25            2.0   
34    10986         11    $21.00        30      0.00            5.0   
35    10989         11    $21.00        15      0.00           30.0   
36    11043         11    $21.00        10      0.00           15.0   
37    11073         11    $21.00        10      0.00           10.0   

    quantity_ahead  q_diff_fr_prev  q_diff_fr_ahead  
0             12.0             NaN              0.0  
1             50.0             0.0            -38.0  
2             12.0            38.0             38.0  
3             24.0           -38.0            -12.0  
4             30.0            12.0             -6.0  
5              6.0             6.0             24.0  
6             30.0           -24.0            -24.0  
7              6.0            24.0             24.0  
8             10.0           -24.0             -4.0  
9              5.0             4.0              5.0  
10            15.0            -5.0            -10.0  
11             3.0            10.0             12.0  
12            50.0           -12.0            -47.0  
13            15.0            47.0             35.0  
14            10.0           -35.0              5.0  
15            15.0            -5.0             -5.0  
16            35.0             5.0            -20.0  
17            15.0            20.0             20.0  
18            14.0           -20.0              1.0  
19            10.0            -1.0              4.0  
20            15.0            -4.0             -5.0  
21             5.0             5.0            

In [21]:
# Alternate way to calculate difference (without a subquery)
sql_query = """
SELECT *,
       (quantity - LAG(quantity, 1) OVER(ORDER BY orderid)) AS quantity_prev,
       (quantity - LEAD(quantity, 1) OVER(ORDER BY orderid)) AS quantity_ahead
FROM orderdetails
WHERE productid=11;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount  quantity_prev  \
0     10248         11    $14.00        12      0.00            NaN   
1     10296         11    $16.80        12      0.00            0.0   
2     10327         11    $16.80        50      0.20           38.0   
3     10353         11    $16.80        12      0.20          -38.0   
4     10365         11    $16.80        24      0.00           12.0   
5     10407         11    $16.80        30      0.00            6.0   
6     10434         11    $16.80         6      0.00          -24.0   
7     10442         11    $16.80        30      0.00           24.0   
8     10443         11    $16.80         6      0.20          -24.0   
9     10466         11    $16.80        10      0.00            4.0   
10    10486         11    $16.80         5      0.00           -5.0   
11    10489         11    $16.80        15      0.25           10.0   
12    10528         11    $21.00         3      0.00          -12.0   
13    10535         11    $21.00        50      0.10           47.0   
14    10542         11    $21.00        15      0.05          -35.0   
15    10545         11    $21.00        10      0.00           -5.0   
16    10553         11    $21.00        15      0.00            5.0   
17    10566         11    $21.00        35      0.15           20.0   
18    10570         11    $21.00        15      0.05          -20.0   
19    10614         11    $21.00        14      0.00           -1.0   
20    10637         11    $21.00        10      0.00           -4.0   
21    10698         11    $21.00        15      0.00            5.0   
22    10726         11    $21.00         5      0.00          -10.0   
23    10770         11    $21.00        15      0.25           10.0   
24    10797         11    $21.00        20      0.00            5.0   
25    10800         11    $21.00        50      0.10           30.0   
26    10823         11    $21.00        20      0.10          -30.0   
27    10842         11    $21.00        15      0.00           -5.0   
28    10862         11    $21.00        25      0.00           10.0   
29    10869         11    $21.00        10      0.00          -15.0   
30    10889         11    $21.00        40      0.00           30.0   
31    10912         11    $21.00        40      0.25            0.0   
32    10926         11    $21.00         2      0.00          -38.0   
33    10944         11    $21.00         5      0.25            3.0   
34    10986         11    $21.00        30      0.00           25.0   
35    10989         11    $21.00        15      0.00          -15.0   
36    11043         11    $21.00        10      0.00           -5.0   
37    11073         11    $21.00        10      0.00            0.0   

    quantity_ahead  
0              0.0  
1            -38.0  
2             38.0  
3            -12.0  
4             -6.0  
5             24.0  
6            -24.0  
7             24.0  
8             -4.0  
9              5.0  
10           -10.0  
11            12.0  
12           -47.0  
13            35.0  
14             5.0  
15            -5.0  
16           -20.0  
17            20.0  
18             1.0  
19             4.0  
20            -5.0  
21            10.0  
22           -10.0  
23            -5.0  
24           -30.0  
25            30.0  
26             5.0  
27           -10.0  
28            15.0  
29           -30.0  
30             0.0  
31            38.0  
32            -3.0  
33           -25.0  
34            15.0  
35             5.0  
36             0.0  
37             NaN

In [79]:
# Calculate difference + using partition
sql_query = """
SELECT t.*,
       t.quantity-t.quantity_prev AS q_diff_fr_prev,
       t.quantity-t.quantity_ahead AS q_diff_fr_ahead
FROM
    (SELECT *,
           LAG(quantity, 1) OVER(PARTITION BY productid) AS quantity_prev,
           LEAD(quantity, 1) OVER(PARTITION BY productid) AS quantity_ahead
    FROM orderdetails) AS t;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount  quantity_prev  \
0       10838          1    $18.00         4      0.25            NaN   
1       10911          1    $18.00        10      0.00            4.0   
2       10413          1    $14.40        24      0.00           10.0   
3       10691          1    $18.00        30      0.00           24.0   
4       10646          1    $18.00        15      0.25           30.0   
...       ...        ...       ...       ...       ...            ...   
2150    10590         77    $13.00        60      0.05           20.0   
2151    10946         77    $13.00        40      0.00           60.0   
2152    10399         77    $10.40        14      0.00           40.0   
2153    10812         77    $13.00        20      0.00           14.0   
2154    10290         77    $10.40        10      0.00           20.0   

      quantity_ahead  q_diff_fr_prev  q_diff_fr_ahead  
0               10.0             NaN             -6.0  
1               24.0             6.0            -14.0  
2               30.0            14.0             -6.0  
3               15.0             6.0             15.0  
4               20.0           -15.0             -5.0  
...              ...             ...              ...  
2150            40.0            40.0             20.0  
2151            14.0           -20.0             26.0  
2152            20.0           -26.0             -6.0  
2153            10.0             6.0             10.0  
2154             NaN           -10.0              NaN  

[2155 rows x 9 columns]

**In the next cell, I use a CTE instead of the subquery. It's almost exactly the same in usage.**

In [23]:
# Using a CTE instead of a subquery

sql_query = """
WITH t AS 
    (SELECT *,
           LAG(quantity, 1) OVER(PARTITION BY productid) AS quantity_prev,
           LEAD(quantity, 1) OVER(PARTITION BY productid) AS quantity_ahead
    FROM orderdetails)
    
SELECT t.*,
       t.quantity-t.quantity_prev AS q_diff_fr_prev,
       t.quantity-t.quantity_ahead AS q_diff_fr_ahead
FROM t;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid  productid unitprice  quantity  discount  quantity_prev  \
0       10838          1    $18.00         4      0.25            NaN   
1       10911          1    $18.00        10      0.00            4.0   
2       10413          1    $14.40        24      0.00           10.0   
3       10691          1    $18.00        30      0.00           24.0   
4       10646          1    $18.00        15      0.25           30.0   
...       ...        ...       ...       ...       ...            ...   
2150    10590         77    $13.00        60      0.05           20.0   
2151    10946         77    $13.00        40      0.00           60.0   
2152    10399         77    $10.40        14      0.00           40.0   
2153    10812         77    $13.00        20      0.00           14.0   
2154    10290         77    $10.40        10      0.00           20.0   

      quantity_ahead  q_diff_fr_prev  q_diff_fr_ahead  
0               10.0             NaN             -6.0  
1               24.0             6.0            -14.0  
2               30.0            14.0             -6.0  
3               15.0             6.0             15.0  
4               20.0           -15.0             -5.0  
...              ...             ...              ...  
2150            40.0            40.0             20.0  
2151            14.0           -20.0             26.0  
2152            20.0           -26.0             -6.0  
2153            10.0             6.0             10.0  
2154             NaN           -10.0              NaN  

[2155 rows x 9 columns]

Bonus questions:
- Can you return a moving average of orders?


## Query using CASE

SQL concepts: JOIN, CASE, working with dates, checking query with RANDOM()

Orders from the USA that were placed in the summer months (June, July, August) need to get shipped from Canada. All others can stay the same. Return the customerid, country, orderdate, shipcountry and a new column you're creating called newshipcountry.

In [103]:
# Overview of customers table
sql_query = """
SELECT *
FROM customers
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customerid                         companyname     contactname  \
0      ALFKI                 Alfreds Futterkiste    Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados    Ana Trujillo   
2      ANTON              Antonio Moreno Taquera  Antonio Moreno   

           contacttitle                       address        city region  \
0  Sales Representative                 Obere Str. 57      Berlin   None   
1                 Owner  Avda. de la Constitucin 2222  Mxico D.F.   None   
2                 Owner               Mataderos  2312  Mxico D.F.   None   

  postalcode  country         phone           fax  
0      12209  Germany   030-0074321   030-0076545  
1      05021   Mexico  (5) 555-4729  (5) 555-3745  
2      05023   Mexico  (5) 555-3932          None

In [105]:
# Overview of ship table
sql_query = """
SELECT *
FROM orders
LIMIT 3;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

orderid customerid  employeeid           orderdate requireddate  \
0    10248      VINET           5 2014-07-04 08:00:00   2014-08-01   
1    10249      TOMSP           6 2014-07-05 04:00:00   2014-08-16   
2    10250      HANAR           4 2014-07-08 15:00:00   2014-08-05   

  shippeddate  shipvia freight                   shipname         shipaddress  \
0  2014-07-16        3  $32.38  Vins et alcools Chevalier  59 rue de l'Abbaye   
1  2014-07-10        1  $11.61          Toms Spezialitten       Luisenstr. 48   
2  2014-07-12        2  $65.83              Hanari Carnes      Rua do Pao, 67   

         shipcity shipregion shippostalcode shipcountry  
0           Reims       None          51100      France  
1          Mnster       None          44087     Germany  
2  Rio de Janeiro         RJ      05454-876      Brazil

In [116]:
# Limit to US orders
sql_query = """
SELECT c.customerid,
       c.country,
       o.oarderdate,
       o.shipcountry
FROM customers AS c
JOIN orders AS o
ON c.customerid=o.customerid
WHERE c.country='USA';
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customerid country           orderdate shipcountry
0        RATTC     USA 2014-07-22 19:00:00         USA
1        WHITC     USA 2014-07-31 00:00:00         USA
2        SPLIR     USA 2014-08-01 05:00:00         USA
3        RATTC     USA 2014-08-02 03:00:00         USA
4        RATTC     USA 2014-08-30 05:00:00         USA
..         ...     ...                 ...         ...
117      GREAL     USA 2016-04-22 13:00:00         USA
118      GREAL     USA 2016-04-30 00:00:00         USA
119      SAVEA     USA 2016-05-01 01:00:00         USA
120      WHITC     USA 2016-05-01 12:00:00         USA
121      RATTC     USA 2016-05-06 17:00:00         USA

[122 rows x 4 columns]

In [132]:
# Limit to US orders + CASE statement for the date
sql_query = """
SELECT c.customerid,
       c.country,
       o.orderdate,
       o.shipcountry,
       CASE WHEN o.orderdate BETWEEN '2014-06-01' AND '2014-08-31'
                 THEN 'Canada'
            ELSE o.shipcountry
            END AS newshipcountry
FROM customers AS c
JOIN orders AS o
ON c.customerid=o.customerid
WHERE c.country='USA';
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customerid country           orderdate shipcountry newshipcountry
0        RATTC     USA 2014-07-22 19:00:00         USA         Canada
1        WHITC     USA 2014-07-31 00:00:00         USA         Canada
2        SPLIR     USA 2014-08-01 05:00:00         USA         Canada
3        RATTC     USA 2014-08-02 03:00:00         USA         Canada
4        RATTC     USA 2014-08-30 05:00:00         USA         Canada
..         ...     ...                 ...         ...            ...
117      GREAL     USA 2016-04-22 13:00:00         USA            USA
118      GREAL     USA 2016-04-30 00:00:00         USA            USA
119      SAVEA     USA 2016-05-01 01:00:00         USA            USA
120      WHITC     USA 2016-05-01 12:00:00         USA            USA
121      RATTC     USA 2016-05-06 17:00:00         USA            USA

[122 rows x 5 columns]

In [133]:
# Limit to US orders + CASE statement for the summer months
sql_query = """
SELECT c.customerid,
       c.country,
       o.orderdate,
       o.shipcountry,
       CASE WHEN EXTRACT (MONTH FROM o.orderdate) BETWEEN 6 AND 8
                 THEN 'Canada'
            ELSE o.shipcountry
            END AS newshipcountry
FROM customers AS c
JOIN orders AS o
ON c.customerid=o.customerid
WHERE c.country='USA';
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customerid country           orderdate shipcountry newshipcountry
0        RATTC     USA 2014-07-22 19:00:00         USA         Canada
1        WHITC     USA 2014-07-31 00:00:00         USA         Canada
2        SPLIR     USA 2014-08-01 05:00:00         USA         Canada
3        RATTC     USA 2014-08-02 03:00:00         USA         Canada
4        RATTC     USA 2014-08-30 05:00:00         USA         Canada
..         ...     ...                 ...         ...            ...
117      GREAL     USA 2016-04-22 13:00:00         USA            USA
118      GREAL     USA 2016-04-30 00:00:00         USA            USA
119      SAVEA     USA 2016-05-01 01:00:00         USA            USA
120      WHITC     USA 2016-05-01 12:00:00         USA            USA
121      RATTC     USA 2016-05-06 17:00:00         USA            USA

[122 rows x 5 columns]

In [140]:
# Limit to US orders + CASE statement for the summer months - check with ORDER BY RANDOM()
sql_query = """
SELECT c.customerid,
       c.country,
       o.orderdate,
       o.shipcountry,
       CASE WHEN EXTRACT (MONTH FROM o.orderdate) BETWEEN 6 AND 8
                 THEN 'Canada'
            ELSE o.shipcountry
            END AS newshipcountry
FROM customers AS c
JOIN orders AS o
ON c.customerid=o.customerid
WHERE c.country='USA'
ORDER BY RANDOM();
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

customerid country           orderdate shipcountry newshipcountry
0        GREAL     USA 2016-03-09 00:00:00         USA            USA
1        THEBI     USA 2016-04-01 01:00:00         USA            USA
2        SAVEA     USA 2016-05-01 01:00:00         USA            USA
3        LONEP     USA 2014-09-30 00:00:00         USA            USA
4        LETSS     USA 2016-02-12 08:00:00         USA            USA
..         ...     ...                 ...         ...            ...
117      SAVEA     USA 2015-11-27 02:00:00         USA            USA
118      OLDWO     USA 2015-07-09 17:00:00         USA         Canada
119      THECR     USA 2016-04-06 03:00:00         USA            USA
120      RATTC     USA 2016-01-26 10:00:00         USA            USA
121      SAVEA     USA 2016-04-17 15:00:00         USA            USA

[122 rows x 5 columns]

In [91]:
# Look at all unique countries
sql_query = """
SELECT DISTINCT(country)
FROM customers;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

country
0     Argentina
1         Spain
2   Switzerland
3         Italy
4     Venezuela
5       Belgium
6        Norway
7        Sweden
8           USA
9        France
10       Mexico
11       Brazil
12      Austria
13       Poland
14           UK
15      Ireland
16      Germany
17      Denmark
18       Canada
19      Finland
20     Portugal

In [ ]:
# Look at all unique shipcountries
sql_query = """
SELECT DISTINCT(country)
FROM customers;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

In [95]:
# How many orders come from the US, mexico, Canada... 

# Using Group by
sql_query = """
SELECT country, COUNT(*)
FROM customers
GROUP BY country
HAVING country IN ('USA', 'Mexico', 'Canada');
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

country  count
0  Mexico      5
1  Canada      3
2     USA     13

In [102]:

# Using Group by
sql_query = """
SELECT country,
       (CASE WHEN country='USA' THEN 1
                  WHEN country='Mexico' THEN 1
                  WHEN country='Canada' THEN 1 END) AS order_counts
FROM customers;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

country  order_counts
0   Germany           NaN
1    Mexico           1.0
2    Mexico           1.0
3        UK           NaN
4    Sweden           NaN
..      ...           ...
86  Finland           NaN
87   Brazil           NaN
88      USA           1.0
89  Finland           NaN
90   Poland           NaN

[91 rows x 2 columns]

# Practicing Leetcode problems


**Rank scores**

Write a SQL query to rank scores. If there is a tie between two scores, both should have the same ranking. Note that after a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no "holes" between ranks.

| Id | Score |
|---|-----|
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |


## Creating a temporary table

In [143]:
# Creating input table
sql_query = """
WITH  input_table (id, score) AS (VALUES (1,3.50), (2,3.65), (3,4.00), (4,3.85), (5,4.00), (6,3.65))

SELECT *
FROM input_table;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

id  score
0   1   3.50
1   2   3.65
2   3   4.00
3   4   3.85
4   5   4.00
5   6   3.65

In [8]:
sql_query = """
WITH 
cohort (id, score) AS (VALUES (1,'Group A'), (2,'Group A'), (3,'Group B'), (4,'Group B')),
event (id, page, click) AS (VALUES (1,'Signup', 1), (2,'Signup', 0), (3,'Signin', 0), (4,'Signin', 0))

SELECT *
FROM event
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

id    page  click
0   1  Signup      1
1   2  Signup      0
2   3  Signin      0
3   4  Signin      0

In [18]:
sql_query = """
WITH 
cohort (id, GroupAssignment) AS (VALUES (1,'Group A'), (2,'Group A'), (3,'Group B'), (4,'Group B')),
event (id, page, click) AS (VALUES (1,'Signup', 1), (2,'Signup', 0), (3,'Signin', 0), (4,'Signin', 0))

SELECT count(ID), GroupAssignment
FROM
    (SELECT DISTINCT E.id, C.GroupAssignment
    FROM Event AS E
    JOIN Cohort AS C
    ON C.id=E.id
    WHERE E.page='Signup') AS sub_table
GROUP BY GroupAssignment

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

count groupassignment
0      2         Group A

In [24]:
sql_query = """
WITH 
cohort (id, GroupAssignment) AS (VALUES (1,'Group A'), (2,'Group A'), (3,'Group B'), (4,'Group B')),
event (id, page, click) AS (VALUES (1,'Signup', 1), (2,'Signup', 0), (3,'Signin', 0), (4,'Signin', 0))

SELECT SUM(E.click)/count(ID) AS Rate, GroupAssignment  
FROM
    (SELECT C.ID, C.GroupAssignment
    FROM Event AS E
    JOIN Cohort AS C
    ON C.ID=E.ID
    WHERE E.Page='Signup') AS sub_table
GROUP BY GroupAssignment
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

DatabaseError: Execution failed on sql '
WITH 
cohort (id, GroupAssignment) AS (VALUES (1,'Group A'), (2,'Group A'), (3,'Group B'), (4,'Group B')),
event (id, page, click) AS (VALUES (1,'Signup', 1), (2,'Signup', 0), (3,'Signin', 0), (4,'Signin', 0))

SELECT SUM(E.click)/count(ID) AS Rate, GroupAssignment  
FROM
    (SELECT C.ID, C.GroupAssignment
    FROM Event AS E
    JOIN Cohort AS C
    ON C.ID=E.ID
    WHERE E.Page='Signup') AS sub_table
GROUP BY GroupAssignment
': missing FROM-clause entry for table "e"
LINE 6: SELECT SUM(E.click)/count(ID) AS Rate, GroupAssignment  
                   ^


## Validating solution with temporary table

In [146]:
# Query with RANK()
sql_query = """
WITH  input_table (id, score) AS (VALUES (1,3.50), (2,3.65), (3,4.00), (4,3.85), (5,4.00), (6,3.65))

SELECT score,
       RANK() OVER(ORDER BY score DESC) AS score_rank
FROM input_table;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

score  score_rank
0   4.00           1
1   4.00           1
2   3.85           3
3   3.65           4
4   3.65           4
5   3.50           6

In [145]:
# Query with dense_rank()
sql_query = """
WITH  input_table (id, score) AS (VALUES (1,3.50), (2,3.65), (3,4.00), (4,3.85), (5,4.00), (6,3.65))

SELECT score,
       DENSE_RANK() OVER(ORDER BY score DESC) AS score_rank
FROM input_table;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

score  score_rank
0   4.00           1
1   4.00           1
2   3.85           2
3   3.65           3
4   3.65           3
5   3.50           4

**Bonus: How would you do this without a window function?**

In [147]:
# Without a window function
sql_query = """
WITH  input_table (id, score) AS (VALUES (1,3.50), (2,3.65), (3,4.00), (4,3.85), (5,4.00), (6,3.65))

SELECT score,
       (SELECT COUNT(DISTINCT(score)) 
        FROM input_table
        WHERE input_table.score >= it.score) AS rank
FROM input_table AS it
ORDER BY rank;
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

score  rank
0   4.00     1
1   4.00     1
2   3.85     2
3   3.65     3
4   3.65     3
5   3.50     4

# -----